In [1]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint

In [2]:
dirrs = ['sentiment/train/', 'sentiment/test/']
sent = []
for dirr in dirrs:
    print dirr
    l = listdir(dirr+'pos')
    print 'pos'
    for r in l:
        t = codecs.open(dirr+'pos/'+r,'r',encoding='utf8').read()
        sent.append(nltk.word_tokenize(t))
    l = listdir(dirr+'neg')
    print 'neg'
    for r in l:
        t = codecs.open(dirr+'neg/'+r,'r',encoding='utf8').read()
        sent.append(nltk.word_tokenize(t))

sentiment/train/
pos
neg
sentiment/test/
pos
neg


In [3]:
m = Word2Vec.load('embedding.model')
data = []
for s in sent:
    a = [m[x] for x in s]
    data.append(a)
data = np.array(data)

In [4]:
def sample(d):
    r = randint(0, data.shape[0] - 1)
    t = d[r]
    x = t[:-1]
    y = t[1:]
    return np.array(x), np.array(y)

s = sample(data)
s[0].shape, s[1].shape

((216, 200), (216, 200))

In [5]:
# hyper-parameters
epochs = 1000
hidden_layer = 150
inp_out_size = 200

In [6]:
'''
debugging 
Wxh = tf.Variable([[1, 2], [3, 4], [5, 6], [7, 8]], dtype=tf.float32)
Whh = tf.Variable([[1, 2], [3, 4]], dtype=tf.float32)
Why = tf.Variable([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
'''
Wxh = tf.Variable(tf.random_uniform([inp_out_size, hidden_layer], maxval=0.01))
Whh = tf.Variable(tf.random_uniform([hidden_layer, hidden_layer], maxval=0.01))
Why = tf.Variable(tf.random_uniform([hidden_layer, inp_out_size], maxval=0.01))

In [7]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    h = tf.nn.tanh(tf.squeeze(tf.matmul(i, Wxh)) + tf.squeeze(tf.matmul(p, Whh)))
    return h

In [13]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
# initial = tf.Variable([1, 2], dtype=tf.float32)
initial = tf.Variable(tf.zeros([hidden_layer]), dtype=tf.float32)
states = tf.scan(recurrence, a, initializer=initial)
outputs = tf.matmul(states, Why)
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(outputs, b))
loss = tf.sqrt(tf.reduce_mean(tf.square(tf.sub(outputs, b))))
trainables = tf.trainable_variables()
grads = tf.gradients(loss, trainables)
grads, _ = tf.clip_by_global_norm(grads, clip_norm=1.0)
grad_var_pairs = zip(grads, trainables)
optimizer = tf.train.AdagradOptimizer(0.01)
optimize_op = optimizer.apply_gradients(grad_var_pairs)

In [14]:
print epochs
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(epochs):
        x, y = sample(data)
        l, s, _ = sess.run([loss, states, optimize_op], {a: x, b: y})
        if i % 100 == 0:
            print 'epoch {0}, loss = {1}'.format(i, l)
    
    x, y = sample(data)
    o, s = sess.run([outputs, states], {a: x, b: y})
    print s

1000
epoch 0, loss = 0.245673522353
epoch 100, loss = 0.249224767089
epoch 200, loss = 0.242077037692
epoch 300, loss = 0.252437114716
epoch 400, loss = 0.238389253616
epoch 500, loss = 0.221532434225
epoch 600, loss = 0.204950749874
epoch 700, loss = 0.186352953315
epoch 800, loss = 0.171733796597
epoch 900, loss = 0.171592190862
[[-0.03228022  0.22836776  0.03262889 ...,  0.17153253 -0.10800488
   0.15492052]
 [-0.0006614   0.28806809  0.09034672 ...,  0.2367128  -0.08148592
   0.23666677]
 [-0.00200164  0.36616492  0.11282786 ...,  0.28415704 -0.10679272
   0.28591082]
 ..., 
 [ 0.06174709  0.44018862  0.19079493 ...,  0.36953259 -0.05254084
   0.37048614]
 [ 0.06106238  0.42486948  0.16782928 ...,  0.37233537 -0.07117874
   0.38437086]
 [ 0.04030007  0.4176141   0.17576316 ...,  0.35387737 -0.09177796
   0.36566627]]


In [20]:
np.array(data[0]).shape

(75, 200)